In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from astropy.modeling import models, fitting
from matplotlib.ticker import FormatStrFormatter
import re
import os
from scipy.optimize import curve_fit
from matplotlib.gridspec import GridSpec
import math
from matplotlib.lines import Line2D
from pathlib import Path
import random

txt_files_folder = '/hpc/home/mf342/PSJ review round 2/Figure 08, Table 3 (Single night and Multiple Night - Parallax versus Find_Orb)'
csv_output_folder = '/hpc/home/mf342/PSJ review round 2/Figure 08, Table 3 (Single night and Multiple Night - Parallax versus Find_Orb)/csv_files'

if not os.path.exists(csv_output_folder):
    os.makedirs(csv_output_folder)
    print(f"Created output folder: {csv_output_folder}")

#columns = ['Date__(UT)__HR:MN', '', '', ' R.A._(ICRF)', 'DEC__(ICRF)', 'delta', 'deldot', '']
columns = [ 'Date__(UT)__HR:MN:SC.fff', 'Date_________JDUT', '', '', 'R.A.___(ICRF)', 'DEC____(ICRF)',  'a-mass', 'mag_ex',             'delta',     'deldot', '']

for file_name in os.listdir(txt_files_folder):
    if file_name.endswith('.txt'):
        file_path = os.path.join(txt_files_folder, file_name)
        start_marker = "$$SOE"
        end_marker = "$$EOE"
        data_lines = []
        try:
            with open(file_path, 'r') as file:
                is_data = False
                for line in file:
                    if start_marker in line:
                        is_data = True
                    elif end_marker in line:
                        is_data = False
                    elif is_data:
                        data_lines.append(line.strip())
            
            data = [line.split(",") for line in data_lines]
            
            max_columns = max(len(row) for row in data)
            data = [row + [''] * (max_columns - len(row)) for row in data]
            
            df = pd.DataFrame(data, columns=columns[:max_columns])
            
            base_name = os.path.splitext(file_name)[0]
            csv_file_path = os.path.join(csv_output_folder, f'{base_name}.csv')
            
            df.to_csv(csv_file_path, index=False)
            print(f"CSV file saved: {csv_file_path}")
        except FileNotFoundError:
            print(f"File not found: {file_path}")
        except Exception as e:
            print(f"Error processing file {file_path}: {e}")

In [ ]:
def read_horizons_file(file_path):
    try:
        data = pd.read_csv(file_path)
        column_mapping = {
            'Date__(UT)__HR:MN': 'JDUT',
            ' R.A._(ICRF)': 'RA_ICRF',  
            'DEC__(ICRF)': 'DEC_ICRF',
            'delta': 'delta'}
        data = data.rename(columns=column_mapping)
        required_columns = ['JDUT', 'RA_ICRF', 'DEC_ICRF', 'delta']
        
        if not all(col in data.columns for col in required_columns):
            raise ValueError(f"Missing one or more required columns in {file_path}")
        return data[required_columns]
    
    except Exception as e:
        print(f"Error reading {file_path}: {e}")
        return pd.DataFrame(columns=['JDUT', 'RA_ICRF', 'DEC_ICRF', 'delta'])

In [ ]:
input_folder  = Path("/hpc/home/mf342/PSJ review round 2/Figure 08, Table 3 (Single night and Multiple Night - Parallax versus Find_Orb)/csv_files")
output_folder = input_folder / "with_uncertainties"

output_folder.mkdir(parents=True, exist_ok=True)
RA_COL  = " R.A._(ICRF)"      
DEC_COL = "DEC__(ICRF)"

SIGMA_ARCSEC = 0.020         
SIGMA_DEG    = SIGMA_ARCSEC / 3600.0  
rng = np.random.default_rng(seed=42)

for csv_file in input_folder.glob("*.csv"):
    df = pd.read_csv(csv_file)
    if RA_COL not in df.columns or DEC_COL not in df.columns:
        print(f"⚠️ Skipping {csv_file.name}, RA/DEC not found")
        continue
    
    ra  = df[RA_COL].to_numpy(dtype=float)
    dec = df[DEC_COL].to_numpy(dtype=float)

    ra_noise  = rng.normal(0.0, SIGMA_DEG, size=len(df))
    dec_noise = rng.normal(0.0, SIGMA_DEG, size=len(df))
    ra_new  = (ra + ra_noise) % 360.0
    dec_new = np.clip(dec + dec_noise, -90.0, 90.0)

    print(f"\n=== {csv_file.name} ===")
    print("Obs |     Original_RA     Original_DEC |    RA_noise(deg)   DEC_noise(deg) |   RA_noise(mas)   DEC_noise(mas) |          New_RA          New_DEC")
    print("----+------------------+----------------+----------------------------------+----------------------------------+---------------------------------")
    for i in range(len(df)):
        print(f"{i+1:3d} | "
              f"{ra[i]:16.8f} {dec[i]:15.8f} | "
              f"{ra_noise[i]:16.8f} {dec_noise[i]:16.8f} | "
              f"{ra_noise[i]*3600*1000:16.2f} {dec_noise[i]*3600*1000:15.2f} | "
              f"{ra_new[i]:16.8f} {dec_new[i]:16.8f}")

    df[RA_COL]  = ra_new
    df[DEC_COL] = dec_new
    out_file = output_folder / csv_file.name
    df.to_csv(out_file, index=False)
    print(f"✅ Saved {out_file}")

In [ ]:
import os, math
import numpy as np
import pandas as pd
from scipy.optimize import curve_fit

# --- 1) Load the truth means from your separate "wide" CSV once ---
def load_true_delta_lookup_from_wide_csv(path_to_wide_csv):
    df = pd.read_csv(path_to_wide_csv)
    numeric = df.select_dtypes(include="number")
    col_means = numeric.mean(numeric_only=True)
    lookup = {}
    for col, val in col_means.items():
        key = col if str(col).endswith(".csv") else f"{col}.csv"
        lookup[key] = float(val)
    return lookup

TRUTH_WIDE_CSV = "/hpc/home/mf342/PSJ review round 2/Figure 08, Table 3 (Single night and Multiple Night - Parallax versus Find_Orb)/PSJ - Paper 1 - Single Night.csv"   # <-- set the correct path
true_delta_lookup = load_true_delta_lookup_from_wide_csv(TRUTH_WIDE_CSV)

# Optional sanity check:
# print("truth keys:", list(true_delta_lookup.keys())[:5])

# --- 2) Your compute function now accepts the lookup as a parameter ---
def compute_distance_in_au_with_quadratic(file_path, true_delta_lookup):
    data = read_horizons_file(file_path)
    
    JD    = data['JDUT'].values
    RA    = data['RA_ICRF'].values
    DEC   = data['DEC_ICRF'].values
    delta = data['delta'].values
    
    DEC_first_row = math.cos(math.radians(DEC[0]))
    mean_delta    = float(np.mean(delta))

    # auto true-delta (Find_Orb) from separate CSV lookup
    fname    = os.path.basename(file_path)
    Find_Orb = true_delta_lookup.get(fname, np.nan)

    RA_min = RA.min()
    RA_normalized = RA - RA_min
    JD_min = JD.min()
    JD_normalized = JD - JD_min

    # ----- Polynomial fit -----
    def fit_function_polynomial(x, A, B, C, D, E):
        return A * np.sin(2 * np.pi * x + B) + C * x + D + E * x**2

    params_poly, covariance_sine = curve_fit(
        fit_function_polynomial, JD_normalized, RA_normalized,
        p0=[1, 0, 0, 0, 0], maxfev=15000
    )
    A, B, C, D, E = params_poly
    A_radians_poly = np.radians(A)
    sine_uncertainties = np.sqrt(np.diag(covariance_sine))
    amplitude_uncertainty_sine = np.radians(sine_uncertainties[0])

    # ----- Linear fit -----
    def fit_function_linear(x, A, B, C, D):
        return A * np.sin(2 * np.pi * x + B) + C * x + D

    params_lin, covariance_linear = curve_fit(
        fit_function_linear, JD_normalized, RA_normalized,
        p0=[1, 0, 0, 0], maxfev=15000
    )
    A_lin, B_lin, C_lin, D_lin = params_lin
    A_lin_radians = np.radians(A_lin)
    linear_uncertainties = np.sqrt(np.diag(covariance_linear))
    amplitude_uncertainty_linear = linear_uncertainties[0]

    # ----- Constants -----
    radius_of_earth_km = 6371
    km_to_au = 149_597_870.7
    latitude_deg = 30.1697
    latitude_rad = math.radians(latitude_deg)

    # ----- Equation 3 -----
    distance_km_poly = radius_of_earth_km / A_radians_poly
    distance_uncertainty_km_poly = distance_km_poly * (amplitude_uncertainty_sine / abs(A_radians_poly))

    distance_km_lin = radius_of_earth_km / A_lin_radians
    distance_uncertainty_km_lin = distance_km_lin * (amplitude_uncertainty_linear / abs(A_lin_radians))

    # ----- Equation 4 -----
    final_distance_km_poly = (distance_km_poly * math.cos(latitude_rad)) / DEC_first_row
    final_distance_uncertainty_km_poly = final_distance_km_poly * (distance_uncertainty_km_poly / distance_km_poly)

    final_distance_km_lin = (distance_km_lin * math.cos(latitude_rad)) / DEC_first_row
    final_distance_uncertainty_km_lin = final_distance_km_lin * (distance_uncertainty_km_lin / distance_km_lin)

    # ----- AU -----
    final_distance_au_poly = final_distance_km_poly / km_to_au
    final_distance_uncertainty_au_poly = final_distance_uncertainty_km_poly / km_to_au

    final_distance_au_lin = final_distance_km_lin / km_to_au
    final_distance_uncertainty_au_lin = final_distance_uncertainty_km_lin / km_to_au

    # ----- Percent differences vs mean_delta -----
    if mean_delta == 0 or final_distance_au_poly == 0:
        percentage_poly = 0.0
        percentage_lin  = 0.0
    else:
        percentage_poly = abs(abs(final_distance_au_poly) - abs(mean_delta)) / ((abs(final_distance_au_poly) + abs(mean_delta)) / 2) * 100.0
        percentage_lin  = abs(abs(final_distance_au_lin)  - abs(mean_delta)) / ((abs(final_distance_au_lin)  + abs(mean_delta))  / 2) * 100.0

    return (
        final_distance_au_poly, final_distance_uncertainty_au_poly,
        final_distance_au_lin,  final_distance_uncertainty_au_lin,
        mean_delta, 
        percentage_poly, percentage_lin,
        params_poly, params_lin, Find_Orb
    )

# --- 3) Pass the lookup into your batch runner too ---
def process_all_asteroid_files(folder_path, true_delta_lookup):
    results = []
    for file_name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file_name)
        if os.path.isfile(file_path) and file_name.endswith('.csv'):
            try:
                result = compute_distance_in_au_with_quadratic(file_path, true_delta_lookup)
                results.append((file_name, *result))

                mean_delta            = result[4]
                distance_in_au_poly   = result[0]
                distance_in_au_lin    = result[2]
                percentage_poly       = result[5]
                percentage_lin        = result[6]
                Find_Orb              = result[9]

                print(f"{file_name:<25} | Mean: {float(mean_delta):>8.4f} | "
                      f"Poly (AU): {float(abs(distance_in_au_poly)):>8.4f} | "
                      f"Poly%: {float(abs(percentage_poly)):>8.4f} | "
                      f"Lin (AU): {float(abs(distance_in_au_lin)):>8.4f} | "
                      f"Lin%: {float(abs(percentage_lin)):>8.4f} | "
                      f"Find_Orb: {float(abs(Find_Orb)) if not np.isnan(Find_Orb) else float('nan'):>8.4f}")

            except Exception as e:
                print(f"Error processing {file_name}: {e}")
    return results

# --- 4) Run ---
asteroids_folder = "/hpc/home/mf342/PSJ review round 2/Figure 08, Table 3 (Single night and Multiple Night - Parallax versus Find_Orb)/csv_files/with_uncertainties"
all_results = process_all_asteroid_files(asteroids_folder, true_delta_lookup)

results_df = pd.DataFrame(all_results, columns=[
    'File Name', 
    'Distance in AU (Poly)', 'Uncertainty (Poly)', 
    'Distance in AU (Lin)',  'Uncertainty (Lin)', 
    'Mean Delta', 
    'percentage_poly', 'percentage_lin',
    'Fit Parameters (Poly)', 'Fit Parameters (Lin)', 'Find_Orb'
])
results_df.to_csv('asteroid_distances_poly_linear.csv', index=False)
print("Results saved successfully!")


In [ ]:
true_values             = abs(results_df['Mean Delta'])
cosine_term_lin_values  = abs(results_df['Distance in AU (Lin)'])
cosine_term_poly_values = abs(results_df['Distance in AU (Poly)'])
Find_Orb                = abs(results_df['Find_Orb'])

In [ ]:
residuals_lin             = (cosine_term_lin_values - true_values)  / true_values
residuals_poly            = (cosine_term_poly_values - true_values) / true_values
residuals_Horizon_FindOrb = (Find_Orb - true_values)                / true_values

asteroid_names = results_df['File Name'].tolist()

residuals_df = pd.DataFrame({
    "Asteroid": asteroid_names,
    "residuals_lin": residuals_lin,
    "residuals_poly": residuals_poly,
    "residuals_Horizon_FindOrb": residuals_Horizon_FindOrb,
    "true_values": true_values})

output_csv = "asteroid_residuals.csv"
residuals_df.to_csv(output_csv, index=False)
print(f"Residuals data saved to {output_csv}")

print(f"{'Asteroid':>20} {'residuals_lin':>20} {'residuals_poly':>20} "
      f"{'residuals_Horizon_FindOrb':>35} {'true_values':>35}")
print("-" * 150)

try:
    iter(true_values)
    true_values_iter = true_values
except TypeError:
    true_values_iter = [true_values] * len(asteroid_names)

for name, r_lin, r_poly, r_hf, tv in zip(
    asteroid_names, residuals_lin, residuals_poly, residuals_Horizon_FindOrb, true_values_iter
):
    print(f"{name:>20} {r_lin:>20f} {r_poly:>20f} {r_hf:>35f} {tv:>35f}")

residuals_Maryann_FindOrb = (Find_Orb - true_values)                / true_values

In [ ]:
sigma_lin                 = np.std(residuals_lin)
sigma_poly                = np.std(residuals_poly)
sigma_Horizon_FindOrb     = np.std(residuals_Horizon_FindOrb)
sigma_Maryann_FindOrb     = np.std(residuals_Maryann_FindOrb)

print(f"sigma_lin: {sigma_lin:.4f}")
print(f"sigma_poly: {sigma_poly:.4f}")
print(f"sigma_Horizons_FindOrb: {sigma_Horizon_FindOrb:.4f}")
print(f"sigma_Maryann_FindOrb: {sigma_Maryann_FindOrb:.4f}")

In [ ]:
lin_mean                 = np.mean(residuals_lin)
poly_mean                = np.mean(residuals_poly)
Horizon_FindOrb_mean     = np.mean(residuals_Horizon_FindOrb)
Maryann_FindOrb_mean     = np.mean(residuals_Maryann_FindOrb)

print(f"lin_mean: {lin_mean:.4f}")
print(f"poly_mean: {poly_mean:.4f}")
print(f"Horizons_FindOrb_mean: {Horizon_FindOrb_mean:.4f}")
print(f"Maryann_FindOrb_mean: {Maryann_FindOrb_mean:.4f}")

In [ ]:
lin_median                 = np.median(residuals_lin)
poly_median                = np.median(residuals_poly)
Horizon_FindOrb_median     = np.median(residuals_Horizon_FindOrb)
Maryann_FindOrb_median     = np.median(residuals_Maryann_FindOrb)

print(f"lin_median: {lin_median:.4f}")
print(f"lin_median: {poly_median:.4f}")
print(f"Horizons_FindOrb_median: {Horizon_FindOrb_median:.4f}")
print(f"Maryann_FindOrb_median: {Maryann_FindOrb_median:.4f}")

In [ ]:
results_df['Lin Percentage Error'] = results_df['percentage_lin'] / 100
results_df['Poly Percentage Error'] = results_df['percentage_poly'] / 100

uncertainty_cosine_lin = abs(results_df['Lin Percentage Error'] * cosine_term_lin_values)
uncertainty_cosine_poly = abs(results_df['Poly Percentage Error'] * cosine_term_poly_values)

residuals_uncertainty_lin = abs(uncertainty_cosine_lin / true_values)
residuals_uncertainty_poly = abs(uncertainty_cosine_poly / true_values)

In [ ]:
# Compute Chi-squared
chi2_lin = np.sum((residuals_lin**2) / (residuals_uncertainty_lin**2))
chi2_poly = np.sum((residuals_poly**2) / (residuals_uncertainty_poly**2))
print(f"Chi-squared (Linear): {chi2_lin:.4f}")
print(f"Chi-squared (Polynomial): {chi2_poly:.4f}")

In [ ]:
# Compute Reduced Chi-squared
N_lin = len(residuals_lin)  
N_poly = len(residuals_poly)

red_chi2_lin = chi2_lin / N_lin
red_chi2_poly = chi2_poly / N_poly

print(f"Reduced Chi-squared (Linear): {red_chi2_lin:.2f}")
print(f"Reduced Chi-squared (Polynomial): {red_chi2_poly:.2f}")

In [ ]:
# STD of asteroids below 0.3 AU
below_03_mask_residuals = (true_values < 0.3)

filtered_file_names = results_df['File Name'][below_03_mask_residuals]
print(filtered_file_names.tolist())

#-------------------------------------------------------------

filtered_residuals_lin = residuals_lin[below_03_mask_residuals]
std_dev_residuals_below_03 = np.std(filtered_residuals_lin)
print(f"Standard Deviation of Linear Fit residuals below 0.3 AU: {std_dev_residuals_below_03:.4f}")

#-------------------------------------------------------------

filtered_residuals_poly = residuals_poly[below_03_mask_residuals]
std_dev_residuals_below_03_poly = np.std(filtered_residuals_poly)
print(f"Standard Deviation of Polynomial Fit residuals below 0.3 AU: {std_dev_residuals_below_03_poly:.4f}")

#-------------------------------------------------------------
filtered_residuals_Horizons_FindOrb = residuals_Horizon_FindOrb[below_03_mask_residuals]
std_dev_residuals_below_03_Horizons_FindOrb = np.std(filtered_residuals_Horizons_FindOrb)
print(f"Standard Deviation of Horizons vs. Find_Orb residuals below 0.3 AU: {std_dev_residuals_below_03_Horizons_FindOrb:.4f}")

#-------------------------------------------------------------

filtered_residuals_Maryann_FindOrb = residuals_Maryann_FindOrb[below_03_mask_residuals]
std_dev_residuals_below_03_Maryann_FindOrb = np.std(filtered_residuals_Maryann_FindOrb)
print(f"Standard Deviation of Maryann vs. Find_Orb residuals below 0.3 AU: {std_dev_residuals_below_03_Maryann_FindOrb:.4f}")


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from astropy.modeling import models, fitting
from matplotlib.ticker import FormatStrFormatter
import re
import os
from scipy.optimize import curve_fit
from matplotlib.gridspec import GridSpec
import math
from matplotlib.lines import Line2D
from pathlib import Path
import random

fig = plt.figure(figsize=(25, 5))
gs = GridSpec(
    1, 3,
    width_ratios=[1, 1, 1],
    height_ratios=[1.5])

colors = plt.cm.tab20(np.linspace(0, 1, len(results_df)))

# ================================================================================================================================

# Bottom-left: Residuals for Linear fit
ax1 = plt.subplot(gs[0, 0])
for i, color in enumerate(colors):
    ax1.errorbar(
        true_values.iloc[i], residuals_lin.iloc[i], fmt='o',
        color=color, markersize=22, markeredgecolor='black', capsize=24, linewidth=2)
ax1.axhline(0, color='red', linestyle='--', linewidth=1, zorder=1)

ax1.text(0.05, 0.84, f"$\\sigma_{{FDS}} = {sigma_lin:.4f}$", fontsize=20, color='blue',
         bbox=dict(facecolor='white', edgecolor='black', alpha=0.7), transform=ax1.transAxes)

ax1.text(0.53, 0.84, f"$mean = {lin_mean:.4f}$", fontsize=20, color='red',
         bbox=dict(facecolor='white', edgecolor='black', alpha=0.7), transform=ax1.transAxes)

ax1.text(0.51, 0.64, f"$median = {lin_median:.4f}$", fontsize=20, color='red',
         bbox=dict(facecolor='white', edgecolor='black', alpha=0.7), transform=ax1.transAxes)
ax1.set_xlim(0.03, 4.5)
ax1.set_xscale('log')
ax1.set_ylim(-0.8, 0.8)
ax1.set_yscale('linear') 
ax1.grid(True, which='both', linestyle='--', linewidth=0.5, alpha=0.7)
ax1.tick_params(axis='both', which='major', labelsize=24)
ax1.set_ylabel(r'$\frac{\mathrm{Equation 1 (au)} - \mathrm{Horizons (au)}}{\mathrm{Horizons (au)}}$', fontsize=24)
ax1.set_xticklabels([])

# Bottom-left: Residuals for Linear fit
ax2 = plt.subplot(gs[0, 2])

for i, color in enumerate(colors):
    ax2.errorbar(true_values.iloc[i], residuals_Horizon_FindOrb.iloc[i], fmt='o', color=color, markersize=22, markeredgecolor='black', capsize=24, linewidth=2)

ax2.axhline(0, color='red', linestyle='--', linewidth=1, zorder=1)

ax2.text(0.05, 0.84, f"$\\sigma_{{FDS}} = {sigma_Horizon_FindOrb:.4f}$", fontsize=20, color='blue', bbox=dict(facecolor='white', edgecolor='black', alpha=0.7), transform=ax2.transAxes)

ax2.text(0.53, 0.84, f"$mean = {Horizon_FindOrb_mean:.4f}$", fontsize=20, color='red',
         bbox=dict(facecolor='white', edgecolor='black', alpha=0.7), transform=ax2.transAxes)

ax2.text(0.51, 0.64, f"$median = {Horizon_FindOrb_median:.4f}$", fontsize=20, color='red',
         bbox=dict(facecolor='white', edgecolor='black', alpha=0.7), transform=ax2.transAxes)

ax2.set_xlim(0.03, 4.5)
ax2.set_xscale('log')
ax2.set_ylim(-0.8, 0.8)
ax2.set_yscale('linear')
ax2.set_yticklabels([])
ax2.set_xticklabels([])
ax2.grid(True, which='both', linestyle='--', linewidth=0.5, alpha=0.7)
ax2.set_ylabel(r'$\frac{\mathrm{Find\_Orb (au)} - \mathrm{Horizons (au)}}{\mathrm{Horizons (au)}}$', fontsize=24)
ax2.tick_params(axis='both', which='major', labelsize=24)

# Bottom-right: Residuals for Polynomial fit
ax3 = plt.subplot(gs[0, 1])
for i, color in enumerate(colors):
    ax3.errorbar(true_values.iloc[i], residuals_poly.iloc[i], fmt='o', color=color, markersize=22, markeredgecolor='black', capsize=24, linewidth=2)
ax3.axhline(0, color='red', linestyle='--', linewidth=1, zorder=1)

ax3.text(0.05, 0.84, f"$\\sigma_{{FDS}} = {sigma_poly:.4f}$", fontsize=20, color='blue',
         bbox=dict(facecolor='white', edgecolor='black', alpha=0.7), transform=ax3.transAxes)

ax3.text(0.53, 0.84, f"$mean = {poly_mean:.4f}$", fontsize=20, color='red',
         bbox=dict(facecolor='white', edgecolor='black', alpha=0.7), transform=ax3.transAxes)

ax3.text(0.51, 0.64, f"$median = {poly_median:.4f}$", fontsize=20, color='red',
         bbox=dict(facecolor='white', edgecolor='black', alpha=0.7), transform=ax3.transAxes)
ax3.set_xticklabels([])
ax3.set_xlim(0.03, 4.5)
ax3.set_xscale('log')
ax3.set_ylim(-0.8, 0.8)
ax3.set_yscale('linear')
ax3.set_yticklabels([])
ax3.set_xticklabels([])
ax3.grid(True, which='both', linestyle='--', linewidth=0.5, alpha=0.7)
ax3.tick_params(axis='both', which='major', labelsize=24)
ax3.set_ylabel(r'$\frac{\mathrm{Equation 5 (au)} - \mathrm{Horizons (au)}}{\mathrm{Horizons (au)}}$', fontsize=24)

# --- Linear residuals ---
ax1.set_title("Residuals: Equation 1 (Linear)", fontsize=24, pad=20)

# --- Polynomial residuals ---
ax3.set_title("Residuals: Equation 5 (Polynomial)", fontsize=24, pad=20)

# --- Find_Orb residuals ---
ax2.set_title("Residuals: Find_Orb vs Horizons", fontsize=24, pad=20)

#----------------------------------------------------------------------------------------------------------
#----------------------------------------------------------------------------------------------------------

# Sorting Asteroid distance in legend
if isinstance(true_values, pd.Series):
    results_df['Mean Delta'] = true_values
else:
    results_df['Mean Delta'] = true_values.mean(axis=1)

sorted_indices = results_df['Mean Delta'].values.argsort()
sorted_file_names = results_df['File Name'].iloc[sorted_indices]
sorted_deltas = results_df['Mean Delta'].iloc[sorted_indices]
colors = np.array(colors)
sorted_colors = colors[sorted_indices]

max_name_length = max(len(file_name.replace('.csv', '')) for file_name in sorted_file_names)
handles = [Line2D([0], [0], marker='o', color='w', markerfacecolor=color,
           markeredgecolor='black', markersize=15,
           label=f"{file_name.replace('.csv', '').ljust(max_name_length)}")
           for file_name, delta, color in zip(sorted_file_names, sorted_deltas, sorted_colors)]

fig.legend(handles=handles, loc='center left', bbox_to_anchor=(0.85, 0.5), fontsize=16,
           title="Asteroids \n (Single nights)", title_fontsize=18, frameon=True)

plt.subplots_adjust(left=0.05, right=0.84, top=0.93, bottom=0.08, wspace=0.20, hspace=0.05)

plt.savefig("Figure 8 (row 1).png", format="png", dpi=300, bbox_inches="tight")
plt.show()


In [ ]:
import os
print("CWD is:", os.getcwd()) 

In [ ]:
import os
import pandas as pd
import unicodedata
from astropy.time import Time
import astropy.units as u
from astropy.coordinates import Angle

# ========== MPC fixed-column layout (1-based) ==========
COL_DATE = 14   # "CYYYY MM DDD.ddddd" (17 chars)
COL_RA   = 32   # "HH MM SS.ss"       (11 chars)
COL_DEC  = 47    # "+DD MM SS.s"       (11 chars)
COL_OBS  = 77      # observatory code    (3 chars)
LINE_LEN = 80
# ======================================================

def _put_at(buf, text, col_start_1based):
    i = col_start_1based - 1
    buf[i:i+len(text)] = list(text)

def _ra_hms(ra_deg, prec=6):
    return Angle(float(ra_deg)*u.deg).to_string(unit=u.hour, sep=' ', precision=prec, pad=True)

def _dec_dms(dec_deg, prec=5):
    s = Angle(float(dec_deg)*u.deg).to_string(unit=u.deg, sep=' ', precision=prec, pad=True, alwayssign=True)
    return ' '.join(s.split())

def _date_from_jd(jd_utc):
    t  = Time(float(jd_utc), format='jd', scale='utc')
    dt = t.to_datetime()
    day_frac = dt.day + (dt.hour + dt.minute/60 + dt.second/3600 + dt.microsecond/1e6/3600)/24.0
    return f"C{dt.year:04d} {dt.month:02d} {day_frac:08.5f}"

def _format_line(jd, ra_deg, dec_deg, observatory='807'):
    line = [' '] * LINE_LEN
    date_str = _date_from_jd(jd)                  # 17 chars
    ra_str   = _ra_hms(ra_deg, prec=6).rjust(11)  # width 11
    dec_str  = _dec_dms(dec_deg, prec=5).rjust(11)# width 11
    obs_str  = f"{observatory:>3}"

    _put_at(line, date_str, COL_DATE)
    _put_at(line, ra_str,   COL_RA)
    _put_at(line, dec_str,  COL_DEC)
    _put_at(line, obs_str,  COL_OBS)
    return ''.join(line)

def _normalize_cols(cols):
    fixed = []
    for c in cols:
        s = unicodedata.normalize('NFKC', str(c))      # normalize unicode
        s = s.replace('\u00A0', ' ')                   # NBSP -> space
        s = ' '.join(s.split())                        # collapse whitespace
        s = s.strip()
        fixed.append(s)
    return fixed

# ---- MAIN ----
formatted_folder  = os.path.join(asteroids_folder, "formatted_obs")
os.makedirs(formatted_folder, exist_ok=True)

for fn in os.listdir(asteroids_folder):
    if not fn.lower().endswith(".csv"):
        continue

    path = os.path.join(asteroids_folder, fn)
    df = pd.read_csv(path)

    # 1) normalize headers and drop blank columns
    df.columns = _normalize_cols(df.columns)
    blank_cols = [c for c in df.columns if c == ""]
    if blank_cols:
        df = df.drop(columns=blank_cols)

    # 2) lock exact column names (after normalization)
    # From your screenshot the canonical headers are:
    # 'Date__(UT)__HR:MN', 'R.A.__(ICRF)', 'DEC__(ICRF)'
    try:
        jd_col  = 'Date__(UT)__HR:MN'
        ra_col  = 'R.A.__(ICRF)'
        dec_col = 'DEC__(ICRF)'

        # if RA still not found, try a space-prefixed fallback
        if ra_col not in df.columns:
            alt = 'R.A.__(ICRF)'  # same text, in case hidden char was removed
            # search by loose match
            candidates = [c for c in df.columns if 'R.A.' in c or 'RA' in c.upper()]
            if candidates:
                ra_col = candidates[0]
            else:
                raise KeyError("RA column not found after normalization.")

        if dec_col not in df.columns:
            candidates = [c for c in df.columns if 'DEC' in c.upper()]
            if candidates:
                dec_col = candidates[0]
            else:
                raise KeyError("DEC column not found after normalization.")

        # 3) coerce to numeric and drop non-numeric rows
        jd  = pd.to_numeric(df[jd_col],  errors="coerce")
        ra  = pd.to_numeric(df[ra_col],  errors="coerce")
        dec = pd.to_numeric(df[dec_col], errors="coerce")
        w   = pd.DataFrame({"jd": jd, "ra": ra, "dec": dec}).dropna()

        if w.empty:
            print(f"⚠️ No numeric rows in {fn}; skipped.")
            continue

        # 4) build lines
        lines = [_format_line(jd, ra, dec, observatory="807")
                 for jd, ra, dec in w[["jd","ra","dec"]].itertuples(index=False, name=None)]

        out = os.path.join(formatted_folder, f"{os.path.splitext(fn)[0]}.obs")
        with open(out, "w") as f:
            f.write("\n".join(lines) + "\n")
        print(f"✅ Saved {out}")

    except Exception as e:
        print(f"⚠️ Error in {fn}: {e}\nColumns seen: {list(df.columns)}")


In [ ]:
# ===================== WRITE FIXED-COLUMN DECIMAL .OBS (Find_Orb) =====================
# Layout:
# col 15: 'C'
# col 16-31: JD (16 chars, 8 decimals)
# col 32-44: RA in decimal degrees, dot at col 36, 8 decimals  -> "dddd.dddddddd" (13 chars)
# col 45-56: Dec in decimal degrees, signed, 8 decimals         -> "+dd.dddddddd" (12 chars)
# col 78-80: Obs code (3 chars)
# Header (per Bill Gray): COM Posn sigma 0.02  (20 mas astrometric uncertainty)

DECIMAL_FIXED_OUT = os.path.join(asteroids_folder, "formatted_obs_decimal_fixed")
os.makedirs(DECIMAL_FIXED_OUT, exist_ok=True)

LINE_LEN = 80
HEADER_LINE = "COM Posn sigma 0.02"  # <-- Bill Gray's recommendation

def _put(buf, s, col1):
    i = col1 - 1
    buf[i:i+len(s)] = list(s)

# -- keep JD as JD (no calendar conversion) --
def _jd_block(jd_utc):
    """Return a 16-character block for JD with 8 decimals (cols 16–31)."""
    return f"{float(jd_utc):>16.8f}"

def _format_ra_deg_32_44(ra_deg):
    """
    RA in decimal degrees occupying cols 32–44 (13 chars total),
    decimal point at col 36 => 4 chars before dot.
    Exactly 8 fractional digits.
    """
    import math
    ra = float(ra_deg) % 360.0
    d_int = int(math.floor(ra))
    frac  = ra - d_int
    frac_int = int(round(frac * 10**8))  # 8 decimals
    if frac_int == 10**8:
        d_int += 1
        frac_int = 0
        if d_int == 360:
            d_int = 0
    return f"{d_int:>4d}.{frac_int:08d}"  # 13 chars, dot at global col 36

def _format_dec_deg_45_56(dec_deg):
    """
    Dec in decimal degrees occupying cols 45–56 (12 chars),
    signed, 8 decimals: '+dd.dddddddd'
    """
    import math
    dec = float(dec_deg)
    sign = '+' if dec >= 0 else '-'
    a = abs(dec)
    d_int = int(math.floor(a))
    frac  = a - d_int
    frac_int = int(round(frac * 10**8))
    if frac_int == 10**8:
        d_int += 1
        frac_int = 0
        if d_int > 90:
            d_int = 90
    return f"{sign}{d_int:02d}.{frac_int:08d}"  # 12 chars

def format_line_decimal_fixed(jd, ra_deg, dec_deg, observatory="807"):
    buf = [' '] * LINE_LEN
    _put(buf, 'C', 15)                             # col 15
    _put(buf, _jd_block(jd), 16)                   # col 16–31  (JD kept as JD)
    _put(buf, _format_ra_deg_32_44(ra_deg), 32)    # col 32–44  (dot at 36)
    _put(buf, _format_dec_deg_45_56(dec_deg), 45)  # col 45–56
    _put(buf, f"{observatory:>3}", 78)             # col 78–80
    return ''.join(buf)

def _already_has_sigma_header(path):
    """Return True if an existing file already contains the exact header line."""
    try:
        with open(path, "r") as f:
            for line in f:
                if line.strip() == HEADER_LINE:
                    return True
    except FileNotFoundError:
        pass
    return False

# --- Use it alongside your CSV loop ---
for fn in os.listdir(asteroids_folder):
    if not fn.lower().endswith(".csv"):
        continue

    path = os.path.join(asteroids_folder, fn)
    df = pd.read_csv(path)

    # reuse your normalization helpers
    df.columns = _normalize_cols(df.columns)
    blank_cols = [c for c in df.columns if c == ""]
    if blank_cols:
        df = df.drop(columns=blank_cols)

    jd_col  = 'Date__(UT)__HR:MN'
    ra_col  = 'R.A.__(ICRF)'
    dec_col = 'DEC__(ICRF)'

    if ra_col not in df.columns:
        cand = [c for c in df.columns if 'R.A' in c or 'RA' in c.upper()]
        ra_col = cand[0] if cand else ra_col
    if dec_col not in df.columns:
        cand = [c for c in df.columns if 'DEC' in c.upper()]
        dec_col = cand[0] if cand else dec_col

    jd  = pd.to_numeric(df.get(jd_col),  errors="coerce")
    ra  = pd.to_numeric(df.get(ra_col),  errors="coerce")
    dec = pd.to_numeric(df.get(dec_col), errors="coerce")
    w   = pd.DataFrame({"jd": jd, "ra": ra, "dec": dec}).dropna()
    if w.empty:
        print(f"⚠️ No numeric rows for fixed decimal output in {fn}; skipped.")
        continue

    lines_fixed = [
        format_line_decimal_fixed(jd, ra, dec, observatory="807")
        for jd, ra, dec in w[["jd", "ra", "dec"]].itertuples(index=False, name=None)
    ]

    out_fixed = os.path.join(
        DECIMAL_FIXED_OUT,
        f"{os.path.splitext(fn)[0]}_decimal_fixed.obs"
    )

    # Compose final contents with the header up front (exactly once)
    contents = []
    if not _already_has_sigma_header(out_fixed):
        contents.append(HEADER_LINE)
    contents.extend(lines_fixed)
    contents.append("")  # final newline

    with open(out_fixed, "w") as f:
        f.write("\n".join(contents))

    print(f"✅ Saved fixed-column decimal file with sigma header: {out_fixed}")
